In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
214,AFG,Asia,Afghanistan,2020-08-11,37054.0,0.0,1312.0,0.0,951.852,0.000,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
370,ALB,Europe,Albania,2020-08-11,6536.0,125.0,200.0,1.0,2271.179,43.436,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
590,DZA,Africa,Algeria,2020-08-11,35712.0,498.0,1312.0,10.0,814.393,11.357,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-08,France,199276.0,0.0,30354.0,0.0,0.0,0.0
0,2020-08-08,China,88685.0,0.0,4688.0,0.0,0.0,0.0
0,2020-08-08,Italy,250225.0,0.0,35218.0,0.0,0.0,0.0
0,2020-08-08,Spain,317811.0,0.0,28513.0,0.0,0.0,0.0
0,2020-08-08,United States,5024317.0,0.0,163019.0,0.0,0.0,0.0
0,2020-08-08,World,19724207.0,0.0,728432.0,0.0,0.0,0.0
0,2020-08-08,United Kingdom,310122.0,0.0,46605.0,0.0,0.0,0.0
0,2020-08-08,Germany,216401.0,0.0,9208.0,0.0,0.0,0.0
0,2020-08-08,Iran,326360.0,0.0,18271.0,0.0,0.0,0.0
0,2020-08-08,Turkey,239483.0,0.0,5826.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-09,France,199276.0,197921.0,30354.0,30324.0,1355.0,30.0
0,2020-08-09,China,88685.0,88672.0,4688.0,4681.0,13.0,7.0
0,2020-08-09,Italy,250225.0,250103.0,35218.0,35203.0,122.0,15.0
0,2020-08-08,Spain,317811.0,314362.0,28513.0,28503.0,3449.0,10.0
0,2020-08-09,United States,5024317.0,4998017.0,163019.0,162425.0,26300.0,594.0
0,2020-08-09,World,19724207.0,19624044.0,728432.0,726953.0,100163.0,1479.0
0,2020-08-09,United Kingdom,310122.0,309763.0,46605.0,46566.0,359.0,39.0
0,2020-08-09,Germany,216401.0,215891.0,9208.0,9196.0,510.0,12.0
0,2020-08-09,Iran,326360.0,324692.0,18271.0,18264.0,1668.0,7.0
0,2020-08-09,Turkey,239483.0,239622.0,5826.0,5829.0,-139.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")